In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [3]:
df = pd.read_csv("/Users/erion/Desktop/MSCF2/ADA/Capstone-Project/data/processed/modeling_dataset.csv", index_col=0, parse_dates=True)

print(df.shape)
df.head()

(5469, 43)


,gold,eurusd,treasury_10y,spy,vix,dxy,oil,target,gold_lag1,gold_lag2,...,vix_lag3,dxy_lag1,dxy_lag2,dxy_lag3,oil_lag1,oil_lag2,oil_lag3,treasury_10y_lag1,treasury_10y_lag2,treasury_10y_lag3
Date,,,,,,,,,,,,,,,,,,,,,
2004-01-13,-0.006099,0.003529,4.028,-0.005830,0.072533,-0.000468,-0.008353,-0.004720,-0.000235,0.005660,...,0.007097,0.005170,-0.005376,-0.007309,0.011950,0.009712,0.010708,4.083,4.086,4.249
2004-01-14,-0.004720,-0.010551,3.986,0.008351,-0.071508,0.006081,0.002033,-0.031539,-0.006099,-0.000235,...,0.073030,-0.000468,0.005170,-0.005376,-0.008353,0.011950,0.009712,4.028,4.083,4.086
2004-01-15,-0.031539,-0.003957,3.971,0.002468,-0.071045,0.006161,-0.030725,-0.004163,-0.004720,-0.006099,...,0.004179,0.006081,-0.000468,0.005170,0.002033,-0.008353,0.011950,3.986,4.028,4.083
2004-01-16,-0.004163,-0.016105,4.014,0.003955,-0.035990,0.013517,0.048744,0.000000,-0.031539,-0.004720,...,0.072533,0.006161,0.006081,-0.000468,-0.030725,0.002033,-0.008353,3.971,3.986,4.028
2004-01-19,0.000000,-0.003546,4.014,0.000000,0.000000,-0.000798,0.000000,0.014753,-0.004163,-0.031539,...,-0.071508,0.013517,0.006161,0.006081,0.048744,-0.030725,0.002033,4.014,3.971,3.986


In [4]:
train = df.loc[:'2015-12-31'].copy()
test  = df.loc['2016-01-01':'2016-12-31'].copy()

X_train = train.drop(columns=['target'])
y_train = train['target']

X_test = test.drop(columns=['target'])
y_test = test['target']

print("Train samples:", len(X_train))
print("Test samples:", len(X_test))

Train samples: 3123
Test samples: 261


In [5]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)  # Output: predicted return
])

model.compile(
    optimizer='adam',
    loss='mse'
)

model.summary()

/opt/anaconda3/envs/goldcapstone/lib/python3.10/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         2,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,865 (19.00 KB)

 Trainable params: 4,865 (19.00 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=32,
    verbose=1
)

Epoch 1/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.2453 - val_loss: 0.0518
Epoch 2/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 682us/step - loss: 0.0473 - val_loss: 0.0322
Epoch 3/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 673us/step - loss: 0.0278 - val_loss: 0.0235
Epoch 4/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 674us/step - loss: 0.0189 - val_loss: 0.0194
Epoch 5/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 671us/step - loss: 0.0141 - val_loss: 0.0168
Epoch 6/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 693us/step - loss: 0.0114 - val_loss: 0.0145
Epoch 7/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 843us/step - loss: 0.0091 - val_loss: 0.0127
Epoch 8/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 758us/step - loss: 0.0074 - val_loss: 0.0117
Epoch 9/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 747us/step - loss: 0.0063 - val_loss: 0.0108
Epoch 10/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 768us/step - loss: 0.0054 - val_loss: 0.0102
Epoch 11/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 735us/step - loss: 0.0048 - val_loss: 0.0096
Epoch 12/50
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 732us/step -

In [8]:
mlp_pred = model.predict(X_test_scaled).flatten()

rmse_mlp = np.sqrt(mean_squared_error(y_test, mlp_pred))
mae_mlp = mean_absolute_error(y_test, mlp_pred)
dir_acc_mlp = (np.sign(mlp_pred) == np.sign(y_test)).mean() * 100

print("MLP Performance")
print("---------------------------")
print(f"RMSE: {rmse_mlp:.6f}")
print(f"MAE:  {mae_mlp:.6f}")
print(f"Directional Accuracy: {dir_acc_mlp:.2f}%")

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
MLP Performance
---------------------------
RMSE: 0.051043
MAE:  0.039661
Directional Accuracy: 47.89%


In [9]:
def create_sequences(X, y, timesteps=5):
    X_seq, y_seq = [], []
    for i in range(len(X) - timesteps):
        X_seq.append(X[i:i+timesteps])
        y_seq.append(y[i+timesteps])
    return np.array(X_seq), np.array(y_seq)

# Convert scaled features to sequences
timesteps = 5

X_train_lstm, y_train_lstm = create_sequences(X_train_scaled, y_train.values, timesteps)
X_test_lstm, y_test_lstm   = create_sequences(X_test_scaled, y_test.values, timesteps)

print("LSTM train shape:", X_train_lstm.shape)
print("LSTM test shape:", X_test_lstm.shape)

LSTM train shape: (3118, 5, 42)
LSTM test shape: (256, 5, 42)


In [10]:
from tensorflow.keras.layers import LSTM

lstm_model = Sequential([
    LSTM(64, activation='tanh', return_sequences=False, input_shape=(timesteps, X_train_lstm.shape[2])),
    Dense(32, activation='relu'),
    Dense(1)
])

lstm_model.compile(
    optimizer='adam',
    loss='mse'
)

lstm_model.summary()

/opt/anaconda3/envs/goldcapstone/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 64)             │        27,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,505 (115.25 KB)

 Trainable params: 29,505 (115.25 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
history_lstm = lstm_model.fit(
    X_train_lstm, y_train_lstm,
    validation_split=0.2,
    epochs=40,
    batch_size=32,
    verbose=1
)

Epoch 1/40
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0106 - val_loss: 0.0030
Epoch 2/40
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0021 - val_loss: 0.0023
Epoch 3/40
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0012 - val_loss: 0.0022
Epoch 4/40
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.4353e-04 - val_loss: 0.0021
Epoch 5/40
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.4729e-04 - val_loss: 0.0016
Epoch 6/40
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 5.2073e-04 - val_loss: 0.0015
Epoch 7/40
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.4692e-04 - val_loss: 0.0014
Epoch 8/40
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.7730e-04 - val_loss: 0.0016
Epoch 9/40
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.6858e-04 - val_loss: 0.0013
Epoch 10/40
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3.7504e-04 - val_loss: 0.0014
Epoch 11/40
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.7692e-04 - val_loss: 0.0014
Epoch 12/40
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [12]:
lstm_pred = lstm_model.predict(X_test_lstm).flatten()

rmse_lstm = np.sqrt(mean_squared_error(y_test_lstm, lstm_pred))
mae_lstm = mean_absolute_error(y_test_lstm, lstm_pred)
dir_acc_lstm = (np.sign(lstm_pred) == np.sign(y_test_lstm)).mean() * 100

print("LSTM Performance")
print("---------------------------")
print(f"RMSE: {rmse_lstm:.6f}")
print(f"MAE:  {mae_lstm:.6f}")
print(f"Directional Accuracy: {dir_acc_lstm:.2f}%")

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
LSTM Performance
---------------------------
RMSE: 0.029928
MAE:  0.024555
Directional Accuracy: 42.97%


In [13]:
from tensorflow.keras.layers import GRU

gru_model = Sequential([
    GRU(64, activation='tanh', return_sequences=False, input_shape=(timesteps, X_train_lstm.shape[2])),
    Dense(32, activation='relu'),
    Dense(1)
])

gru_model.compile(
    optimizer='adam',
    loss='mse'
)

gru_model.summary()

/opt/anaconda3/envs/goldcapstone/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ gru (GRU)                       │ (None, 64)             │        20,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,849 (89.25 KB)

 Trainable params: 22,849 (89.25 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
history_gru = gru_model.fit(
    X_train_lstm, y_train_lstm,
    validation_split=0.2,
    epochs=40,
    batch_size=32,
    verbose=1
)

Epoch 1/40
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0217 - val_loss: 0.0049
Epoch 2/40
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0039 - val_loss: 0.0035
Epoch 3/40
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0023 - val_loss: 0.0029
Epoch 4/40
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0016 - val_loss: 0.0026
Epoch 5/40
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0012 - val_loss: 0.0022
Epoch 6/40
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 9.4398e-04 - val_loss: 0.0020
Epoch 7/40
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.7689e-04 - val_loss: 0.0018
Epoch 8/40
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.4055e-04 - val_loss: 0.0018
Epoch 9/40
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.6063e-04 - val_loss: 0.0016
Epoch 10/40
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 7.0195e-04 - val_loss: 0.0015
Epoch 11/40
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 6.9149e-04 - val_loss: 0.0013
Epoch 12/40
78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

In [15]:
gru_pred = gru_model.predict(X_test_lstm).flatten()

rmse_gru = np.sqrt(mean_squared_error(y_test_lstm, gru_pred))
mae_gru = mean_absolute_error(y_test_lstm, gru_pred)
dir_acc_gru = (np.sign(gru_pred) == np.sign(y_test_lstm)).mean() * 100

print("GRU Performance")
print("---------------------------")
print(f"RMSE: {rmse_gru:.6f}")
print(f"MAE:  {mae_gru:.6f}")
print(f"Directional Accuracy: {dir_acc_gru:.2f}%")

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
GRU Performance
---------------------------
RMSE: 0.027986
MAE:  0.021664
Directional Accuracy: 42.19%


In [16]:
results_dl = pd.DataFrame({
    'Model': ['MLP', 'LSTM', 'GRU'],
    'RMSE': [rmse_mlp, rmse_lstm, rmse_gru],
    'MAE': [mae_mlp, mae_lstm, mae_gru],
    'Directional Accuracy (%)': [dir_acc_mlp, dir_acc_lstm, dir_acc_gru]
})

print("\nDEEP LEARNING MODEL COMPARISON (2016 Test Set)")
print(results_dl)


DEEP LEARNING MODEL COMPARISON (2016 Test Set)
  Model      RMSE       MAE  Directional Accuracy (%)
0   MLP  0.051043  0.039661                  47.89272
1  LSTM  0.029928  0.024555                  42.96875
2   GRU  0.027986  0.021664                  42.18750
